These are our imports

In [ ]:
#imports
import numpy as np
print("numpy imported")
import json
print("json imported")
import time
print("time imported")
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
print("tensorflow imported") 
from keras import Sequential
print("keras imported")
import config
channelID,serverID,targetID = config.getScope()
print("config imported")
import pickle
print("pickle imported")
import random
print("random imported")
import string
print("string imported")

Run the section below to process your data

In [ ]:
channelID,serverID,targetID = config.getScope()
daTestDoc = open("TestDoc","r",encoding="utf-8")
daMessageList = eval(daTestDoc.read())
with open('message.list','wb') as messageList_file:
    pickle.dump(daMessageList,messageList_file)
    print(":flushed:")

In [ ]:
inputList,outputList = [],[]
with open("message.list",'rb') as daMessageList:
    channelID,serverID,targetID = config.getScope()
    daMessageList = pickle.load(daMessageList)
    for i in range(len(daMessageList)):
        if i != len(daMessageList)+1:
            message = daMessageList[i]
            if message["author"]["id"] == str(targetID):
                #if i != 0:
                if True:
                    previousMessage = daMessageList[i-1]
                    if previousMessage["author"]["id"] == str(targetID):
                        if len(outputList) != 0:    
                            outputList[len(outputList)-1] += "   " + message["content"]
                    else:
                        if type(message.get("referenced_message"))!=dict:
                            outputList.append(message["content"])
                            message = daMessageList[i+1]
                            inputList.append(message["content"])
                        else:
                            inputList.append(message["referenced_message"]["content"])
                            outputList.append(message["content"])

        myIndice = random.randint(0,len(inputList))

    # for message in range(len(inputList)):
    # 	inputList[message] = removePunctuation(removeCaps(inputList[message]))
    with open('input.list','wb') as inputList_file:
        pickle.dump(inputList,inputList_file)
    with open('output.list','wb') as outputList_file:
        pickle.dump(outputList,outputList_file)
    print("Complete! (1/2)")


with open('input.list','rb') as inputList_file:
    with open('output.list','rb') as outputList_file:
        outputList = pickle.load(outputList_file)
        inputList = pickle.load(inputList_file)
        
        removePunctuation = lambda x: ''.join(char for char in x if not char in string.punctuation)
        removeCaps = lambda x: x.lower()

        for message in range(len(inputList)):
            inputList[message] = removePunctuation(removeCaps(inputList[message]))
            outputList[message] = removePunctuation(removeCaps(outputList[message]))
        #inputList = np.array(inputList)
        #outputList = np.array(outputList)
        with open('input.list','wb') as inputList_file:
            pickle.dump(inputList,inputList_file)
        with open('output.list','wb') as outputList_file:
            pickle.dump(outputList,outputList_file)

        print("Complete! (2/2)")

In [ ]:
with open('input.list','rb') as inputList_file:
    inputList = pickle.load(inputList_file)
    daRandInt = random.randint(0,len(inputList)-1)
    print(outputList[100:200])

In [ ]:
import gc
del daMessageList
del outputList
del daTestDoc
del inputList
gc.collect()

In [ ]:
daShape = None


with open('input.list','rb') as inputList_file:
    with open('output.list','rb') as outputList_file:
        outputList = pickle.load(outputList_file)
        inputList = pickle.load(inputList_file)

        #outputList = ["hello there","among us in real life", "hello impostor"]
        #inputList = ["hey","im so sus","omg emergency meeting"]

        tokenizer = Tokenizer()
        #print(type(inputList+outputList))
        tokenizer.fit_on_texts(inputList)
        #print(inputList[0:10])
        inputList = tokenizer.texts_to_sequences(inputList)
        outputList = tokenizer.texts_to_sequences(outputList)

        #print(inputList)
        
        inputList = pad_sequences(inputList)
        outputList = pad_sequences(outputList)


        print(inputList.shape)
        print(outputList.shape)
        if inputList.shape[1] > outputList.shape[1]:
            daMaxLen = inputList.shape[1]
        else:
            daMaxLen = outputList.shape[1]
        print(daMaxLen)

        inputList = pad_sequences(inputList, maxlen = daMaxLen)
        outputList = pad_sequences(outputList, maxlen = daMaxLen)

        daShape = outputList.shape

        inputList = np.expand_dims(inputList,axis=0)
        outputList = np.expand_dims(outputList,axis=0)

        inputList = to_categorical(inputList)
        outputList = to_categorical(outputList)

        inputList = inputList.astype(np.uint8)
        outputList = outputList.astype(np.uint8)

        with open('input.list','wb') as inputList_file:
            pickle.dump(inputList,inputList_file)
        with open('output.list','wb') as outputList_file:
            pickle.dump(outputList,outputList_file)
        print("Complete!")

** This is a test to make sure the filters run correctly. **

In [ ]:
with open('input.list','rb') as inputList:
    with open('output.list','rb') as outputList:
        inputList = pickle.load(inputList)
        outputList = pickle.load(outputList)
        for message in range(len(inputList)):
            for letter in ("abcdefghijklmnopqrstuvwxyz".upper()+string.punctuation):
                if letter in inputList[message]:
                    print(inputList[message])
                if letter in outputList[message]:
                    print(outputList[message])
print("!")

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Embedding, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import RMSprop

In [ ]:
#model config
print(inputList.shape)
print(outputList.shape)
randomIndice = random.randint(0,len(inputList)-1)
print(inputList[0][0][0].shape)

dataSize = 2789 #number of features

In [ ]:
dimensionality = 256#The batch size and number of epochs
batch_size = 10
epochs = 600#Encoder
num_encoder_tokens = dataSize
num_decoder_tokens = dataSize
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
rmsprop = RMSprop(learning_rate=0.0001)

model.compile(optimizer = rmsprop, loss = "categorical_crossentropy", metrics = ['accuracy'])
#View model info


In [ ]:
#Inverse tokenization
reverse_word_map = tokenizer.word_index
#print(reverse_word_map.keys())
reverse_word_map = {v: k for k, v in reverse_word_map.items()}
reverse_word_map[0] = ""

#Model

training_model = Model(wordList, wordList, decoder_outputs)#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')#Training
training_model.fit(inputList[0], outputList[0], batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('training_model.h5')

Stats

In [ ]:
model.summary()

In [ ]:
with open('input.list','rb') as inputList:
    with open('output.list','rb') as outputList:
        inputList = pickle.load(inputList)
        outputList = pickle.load(outputList)
        inputWordCounter = 0
        inputCharCounter = 0
        outputWordCounter = 0
        outputCharCounter = 0
        for message in range(len(inputList)):
            for char in inputList[message]:
                if char == " ":
                    inputWordCounter += 1
                inputCharCounter += 1
        for message in range(len(inputList)):
            for char in outputList[message]:
                if char == " ":
                    outputWordCounter += 1
                outputCharCounter += 1
        print("Count of Input Words: " + str(inputWordCounter))
        print("Count of Input Characters: " + str(inputCharCounter))
        print("Count of Output Words: " + str(outputWordCounter))
        print("Count of Output Characters: " + str(outputCharCounter))
